In [14]:
from utils import get_openai_api_key, get_openai_base_url, get_azure_openai_api_key, get_azure_openai_endpoint
# OPENAI_API_KEY = get_openai_api_key()
# OPENAI_BASE_URL = get_openai_base_url()
AZURE_OPENAI_API_KEY = get_azure_openai_api_key()
AZURE_OPENAI_ENDPOINT = get_azure_openai_endpoint()
llm_config={"model": "gpt-4-test", "api_type": "azure", "api_key": "87eb5a0211fa455194900b26c01d3a21", "base_url": "https://chinasnrunlimited.openai.azure.com/", "api_version": "2024-06-01"}

In [15]:
query = "最新一天的储蓄存款时点数？"

In [16]:
task = f"Performing a text-to-sql task in a banking business system requires responding based on the user's query and the information provided in two given tables. The user query now is: {query}"

In [17]:
print(task)


Performing a text-to-sql task in a banking business system requires responding based on the user's query and the information provided in two given tables. The user query now is: 最新一天的储蓄存款时点数？


In [18]:
import autogen

In [19]:
# user_proxy = autogen.ConversableAgent(
#     name="Admin",
#     system_message="Give the query, and send "
#     "instructions to writer to refine the sql statement result.",
#     code_execution_config=False,
#     llm_config=llm_config,
#     human_input_mode="ALWAYS",
# )

In [20]:
database = """
### 数据范围
给定的两张表的表结构如下：

CREATE TABLE f_rpt_mgmt_ssap_index_info (

 ETL_TIME varchar    '跑批时间',

 ORG_TYPE varchar    '机构类型',

 ORG_NO varchar    '机构号',

 YB_DEP decimal '一般性存款时点',

 YB_AVG_DEP decimal '一般性存款时期',

 CZ_DEP decimal '财政性存款时点',

 CZ_AVG_DEP decimal '财政性存款时期',

 TY_DEP decimal '同业存款时点',

 TY_AVG_DEP decimal '同业存款时期',

 CXHQ_DEP decimal '储蓄存款活期时点',

 CXHQ_AVG_DEP decimal '储蓄存款活期时期',

 CXDQ_DEP decimal '储蓄存款定期时点',

 CXDQ_AVG_DEP decimal '储蓄存款定期时期',

 LL_DEP decimal '六类存款时点',

 LL_AVG_DEP decimal '六类存款时期',

 XY_DEP decimal '协议存款时点',

 XY_AVG_DEP decimal '协议存款时期',

 NTY_DEP decimal '非银同业存款时点',

 NTY_AVG_DEP decimal '非银同业存款时期',

 JGX_DEP decimal '结构性存款时点',

 JGX_AVG_DEP decimal '结构性存款时期',

 BB_DEP decimal '保本理财时点',

 BB_AVG_DEP decimal '保本理财时期',

 DE_DEP decimal '大额存单时点',

 DE_AVG_DEP decimal '大额存单时期',

 ZN_DEP decimal '智能存款时点',

 ZN_AVG_DEP decimal '智能存款时期',

 CUST_NUM decimal '个人客户数',

 LSDK_RATE decimal '零售贷款',

 DSCK_RATE decimal '对私存款',

 DEP decimal '存款时点',

 DEP_AVG decimal '存款时期',

 CX_DEP decimal '储蓄存款时点',

 CX_DEP_AVG decimal '储蓄存款时期',

 LOAN decimal '贷款总额时点',

 LOAN_AVG decimal '贷款总额时期',

 LS_LOAN decimal '对私贷款时点',

 DS_LOAN_AVG decimal '对私贷款时期',

 LS_LOAN_AVG decimal '零售贷款时期数储蓄',

 INCOME decimal '非息收入',

 XYK_INCOME decimal '信用卡非息收入',

 CF_INCOME decimal '财富业务非息收入',

 XYK_CNT bigint    '信用卡有效户',

 MGER_ASSET decimal '对私管理资产时点',

 MGER_ASSET_AVG decimal '对私管理资产时期',

 LSKH_CNT bigint    '零售客户开户数',

 TZKH_CNT bigint    '客户分层提质数',

 JCKH_CNT bigint    '交叉销售客户数',

 LOAD_DT varchar    '数据日期',

 ID bigint '自增长ID',

 PRIMARY KEY (ID),

 KEY IDX_LOAD_DT (LOAD_DT) USING BTREE,

 KEY IDX_ORG_NO (ORG_NO) USING BTREE,

 KEY IDX_ORG_TYPE (ORG_TYPE) USING BTREE

) ENGINE=InnoDB AUTO_INCREMENT=376806 DEFAULT CHARSET=utf8 COMMENT='数据家园供自助分析平台指标宽表'

 

CREATE TABLE f_rpt_mgmt_ssap_corp_index_info (

 ETM_TIME varchar    '数据加工时间',

 ORG_TYPE varchar    '机构类型',

 ORG_NO varchar    '机构号',

 PH_BAL decimal '普惠贷款（时点）',

 PH_CRE_BAL decimal '普惠信用贷款（时点）',

 PH_LOAN_CNT decimal '普惠小微型企业首贷户',

 XW_LNE decimal '小微企业贷款无还本续贷放款金额',

 ALL_XW_LNE decimal '符合小微企业无还本续贷条件的还款金额',

 XW_RATE decimal '小微企业贷款无还本续贷率',

 CORP_TZ_POINT decimal '对公客户分层提质积分',

 DPSIT_YAVG_BAL decimal '新增对公存款时期数（调整后）',

 CORP_LOAN_BAL decimal '对公贷款（表内）（时点）',

 PER_LN_BAL decimal '个人经营性贷款（时点）',

 ALL_LDCUR_LN_BAL decimal '涉农贷款（时点）',

 HQ_DEP_AVG decimal '对公存款活期',

 DEP_AVG decimal '对公存款',

 HQ_CORP_RATE decimal '对公存款活期占比',

 CURR_POINT decimal '对公分层提质积分',

 GREEN_BAL decimal '绿色贷款',

 MANU_BAL decimal '制造业中长期贷款',

 DGHQ_BAL decimal '对公存款活期（时点）',

 DG_CNT int    '对公客户数',

 WTDK_BAL decimal '委托贷款（时点）',

 CDHP_BAL decimal '承兑汇票（时点）',

 KCXYZ_BAL decimal '开出信用证（时点）',

 LX_INCOME decimal '（损益）利息收入',

 LX_DG_INCOME decimal '（损益）对公贷款',

 LX_EXPENSE decimal '（损益）利息支出',

 LX_DG_EXPENSE decimal '（损益）对公存款',

 MID_INCOME decimal '（损益）中间业务收入',

 BB_PROFIT decimal '（损益）报表利润',

 TXDK decimal '（损益）贴现+垫款',

 CZX_DEP decimal '（损益）财政性存款',

 YGGLF decimal '（损益）业务及管理费',

 KC_CASH decimal '（损益）库存现金',

 DWDK decimal '单位贷款（时点）',

 PJRZ decimal '票据融资（时点）',

 SUM_INCOME decimal '公司条线非息收入（小计）',

 INV_BANK_BIZ_INCOME decimal '公司条线非息收入（投行）',

 INTER_BIZ_INCOME decimal '公司条线非息收入（国际）',

 OTHER_BIZ_INCOME decimal '公司条线非息收入（其他）',

 CORE_DEP_TOTAL_BAL decimal '对公存款总额（时点）',

 CORE_DEP_AVG_BAL decimal '对公存款时期',

 CX_DEP decimal '储蓄存款时点',

 CX_DEP_AVG decimal '储蓄存款时期',

 CORE_DEP_DQ_BAL decimal '对公存款定期（时点）',

 DEP_BAL decimal '存款总额（时点）',

 DEP_AVG_BAL decimal '存款总额时期',

 LOAD_DT varchar    '数据日期',

 ID bigint '自增长ID',

 PRIMARY KEY (ID),

 KEY IDX_LOAD_DT (LOAD_DT) USING BTREE,

 KEY IDX_ORG_NO (ORG_NO) USING BTREE,

 KEY IDX_ORG_TYPE (ORG_TYPE) USING BTREE

) ENGINE=InnoDB AUTO_INCREMENT=32050 DEFAULT CHARSET=utf8 COMMENT='数据家园供自助分析平台对公指标宽表'

"""

In [21]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a user query, please determine "
    "what table information is needed to complete the task. "
    "Please note that the information will all be retrieved within"
    f" the given two tables are as follows {database}."
    "If the user query is not related to the table information, please respond '请重新提问'."
    "If the user query exceeds the given table information, please respond '超出现有数据范围'."
    "Otherwise, please provide the table information that is needed to be retrieved to complete the text-to-sql task."
    "You must polish your retrieval information based on the feedback you receive and give a refined version.",
    description="Planner. Given a user query and the two original tables, determine what "
    "table information is needed to be retrieved to complete the text-to-sql task. "
    "Also can receive the feedback from Reviewer and then give a refined version.",
    llm_config=llm_config,
)

In [22]:
reviewer = autogen.AssistantAgent(
    name="Reviewer",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a reviewer. You review the retrieved table information from"
                "the planner and judge whether the retrieved information is correct and complete or not according to the user query and the two given tables. "
                "If the retrieved information is not correct and complete enough to complete the text-to-sql task"
                "please provide constructive feedback to help planner improve the quality of the content. "
                "Otherwise, let the planner know that his retrieval information is correct and complete."
                "Finally, if the writer writes the sql statement correctly, please give a positive feedback to the writer.",
)

In [23]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write sql statement in SQL format based on the user query and"
    " table information retrieved provided by the planner"
    "You take feedback from the reviewer and refine your sql statement."
    "If you receive positive feedback from the reviewer, please write the final sql statement."
    "and add 'TERMINATE' at the end of your response to indicate the end of the task.",
    description="Writer."
    "Write sql statement in SQL format based on the user query and"
    " table information retrieved provided by the planner"
    "Take feedback from the reviewer and refine sql statement. ",
)

In [24]:
groupchat = autogen.GroupChat(
    agents=[writer, reviewer, planner],
    messages=[],
    max_round=8,
)

In [25]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


In [26]:
groupchat_result = planner.initiate_chat(
    manager,
    message=task,
)

Planner (to chat_manager):

Performing a text-to-sql task in a banking business system requires responding based on the user's query and the information provided in two given tables. The user query now is: 最新一天的储蓄存款时点数？

--------------------------------------------------------------------------------

Next speaker: Reviewer

Reviewer (to chat_manager):

Sorry, but without seeing the tables and additional background information, I won't be able to evaluate whether the information you retrieved is correct and complete to perform the text-to-sql task. Please provide the tables and the information you retrieved based on the user query ("the latest one-day savings deposit point number" in English). That would be really helpful to carry on with the task and provide constructive feedback.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

Sure, to fulfill the user query ("the lates